_____

In [1]:
import os
import sys
sys.path.append("../utils/")
sys.path.append("../models/")
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from models import UNet, UResNet
# from max_model import ENet3D as ENet
from nn_utils import iterate_minibatches
from mulptiprocessing_utils import par_iterate_minibatches
from pytorch_utils import to_numpy, to_var, loss_cross_entropy, stochastic_step
from data_utils import load_files, random_nonzero_crops, augment, _reshape_to
from data_utils import combine, divide
from metrics import hausdorff, dice
from model_controller import Model_controller

%matplotlib inline

In [2]:
PATH = '../data/Utr/Utrecht/'
PATH1 = '../data/Sing/Singapore/'
PATH2 = '../data/Amst/GE3T/'

In [3]:
masks, t1, flairs, brain_mask = load_files(PATH)
brains = np.concatenate([flairs, t1], axis=1).astype(np.float32)

masks1, t11, flairs1, brain_mask1 = load_files(PATH1)
brains1 = np.concatenate([flairs1, t11], axis=1).astype(np.float32)

masks2, t12, flairs2, brain_mask2 = load_files(PATH2)
brains2 = np.concatenate([flairs2, t12], axis=1).astype(np.float32)

  0%|          | 0/20 [00:00<?, ?it/s]

INFO: brain mask from hardcoded path  ../data/skull_stripping


100%|██████████| 20/20 [00:05<00:00,  3.62it/s]


INFO: data reshape to hardcoded shape new_shape=! (256, 256, 84)


  0%|          | 0/20 [00:00<?, ?it/s]

...

INFO: data reshape to hardcoded shape new_shape=! (256, 256, 84)


  0%|          | 0/20 [00:00<?, ?it/s]

INFO: brain mask from hardcoded path  ../data/skull_stripping


100%|██████████| 20/20 [00:05<00:00,  3.57it/s]


INFO: data reshape to hardcoded shape new_shape=! (256, 256, 84)


_____

____

In [4]:
def get_train_split(get_train_split=20, n=15):
    return np.in1d(np.array(range(get_train_split)), 
                   np.random.choice(range(get_train_split), n, False))

In [5]:
idx1, idx2, idx3 = get_train_split(), get_train_split(), get_train_split()
idx1, idx2, idx3

(array([ True, False,  True, False,  True, False,  True,  True,  True,
         True, False,  True,  True,  True,  True,  True,  True,  True,
        False,  True], dtype=bool),
 array([ True,  True,  True, False,  True, False, False,  True,  True,
        False,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True], dtype=bool),
 array([ True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True, False, False, False,  True,  True,  True,
         True, False], dtype=bool))

X_train = np.concatenate([brains[idx1]])
X_test = np.concatenate([brains[~idx1]])
Y_train = np.concatenate([masks[idx1]])
Y_test = np.concatenate([masks[~idx1]])

___

In [6]:
from sklearn.model_selection import KFold
skf = KFold(n_splits=4, random_state=0, shuffle=True)

In [ ]:
import pickle

In [ ]:
counter=0
for tr_idx, ts_idx in skf.split(range(len(brains)), range(len(masks))):
    counter+=1
    X_train = np.concatenate([brains[tr_idx], brains1[tr_idx], brains2[tr_idx]])
    X_test = np.concatenate([brains[ts_idx], brains1[ts_idx], brains2[ts_idx]])
    Y_train = np.concatenate([masks[tr_idx], masks1[tr_idx], masks2[tr_idx]])
    Y_test = np.concatenate([masks[ts_idx], masks1[ts_idx], masks2[ts_idx]])
    
    model = UNet().cuda()
#     model = UResNet().cuda()
#     model = ENet(3,2).cuda()
    controller = Model_controller(model)
    
    controller.init_train_procedure(batch_size = 30, crops_shape=(52, 52, 40),
                                    context_shape=(52, 52, 40), lr=0.015, 
                                    lr_decayer=lambda x: x*0.75, net='3D',
                                    decay_every_epoch=16, num_of_patches=450)
    
    controller.train(X_train, Y_train, epoches=100, X_val=X_test, y_val=Y_test,
                     hard_negative = lambda epoch: False)
    y = controller.predict(X_test)
    
    np.savez('../reports/figures/UntitledFolder/y_dump_'+str(counter)+'.npy', y)
    
    y = np.logical_and(y[:, 1, ...] > y[:, 0, ...],
                       y[:, 1, ...] > y[:, 2, ...])
    
    for num_img in range(len(Y_test)):
        plt.figure()
        t = np.argmax(np.sum(Y_test[num_img, 0, ...]==1, axis=(-2, -3)))
        plt.subplot(121)
        plt.imshow(y[num_img, ..., t], cmap=plt.cm.binary_r)
        plt.title('dice: ' + str(dice(y[num_img], Y_test[num_img,0]))[:5])
        plt.subplot(122)
        plt.imshow(Y_test[num_img, 0,..., t], cmap=plt.cm.binary_r, vmax=2)
        plt.savefig('../reports/figures/UntitledFolder/'+str(counter)+'_'+str(num_img)+'.png')
        
        plt.figure()
        plt.plot(controller.res['train_loss'], label='train')
        plt.plot(controller.res['val_loss'], label='val')
        plt.legend();
        plt.savefig('../reports/figures/UntitledFolder/'+str(counter)+'_tr_val_3D_UNet.png')
        plt.figure()
        plt.plot(controller.res['lr'], label='lr')
        plt.legend();
        plt.savefig('../reports/figures/UntitledFolder/'+str(counter)+'_lr_3D_UNet.png')
        

0.0422: 100%|██████████| 15/15 [00:34<00:00,  2.26s/it]


0 epoch: 
train loss 0.2910533227026463


0.0880: 100%|██████████| 3/3 [00:04<00:00,  1.40s/it]


val loss 0.1020261471470197
dices [0.14190687361419069, 0.2692340952984924, 0.0, 0.33252812478431915, 0.0030656039239730227, 0.003711558854718982, 0.16965742251223492, 0.050701910336615065, 0.30736483610046827, 0.0, 0.0, 0.19183318853171155, 0.0, 0.017381228273464659, 0.017297297297297298]


0.0907: 100%|██████████| 15/15 [00:36<00:00,  2.36s/it]


...

0.0088: 100%|██████████| 3/3 [00:04<00:00,  1.34s/it]


val loss 0.010955200841029486
dices [0.37764932562620424, 0.77596355292474473, 0.11777777777777777, 0.69760710150521033, 0.16104527499240354, 0.74591886213027314, 0.44365192582025675, 0.54381583233793029, 0.6336151974979729, 0.0, 0.49824561403508771, 0.7410714285714286, 0.25367647058823528, 0.42863827795975668, 0.53204353083434097]


  0%|          | 0/3 [00:00<?, ?it/s]

97 epoch: 
train loss 0.020627392828464507


0.0123: 100%|██████████| 3/3 [00:04<00:00,  1.34s/it]


___

In [ ]:
import pickle
with open('../models/dumps/model_unetcnn.pkl', 'wb') as f:
    pickle.dump(controller.model.state_dict(), f, protocol=pickle.HIGHEST_PROTOCOL)